# Applying the Scientific Method to Scientific Software

Scientific software developers are accustomed to applying the scientific method
to complex research problems, ie., by formulating hypotheses, running experiments,
and analyzing results. The software they build to model, simulate, or analyze
these problems is often just as complex and can benefit from a similar 
methodological approach to ensure correctness and reliability.

In scientific research, confidence is established through rigorous reasoning,
experimentation, and validation against empirical data. Yet the software that
underpins this research is often developed with far less rigor. At best, it typically relies
on conventional unit, integration, and regression tests involving a number of hand-picked
cases that the developers believe are representative of the software's general use cases.
However, these tests often fail to capture the full range of behaviors or edge cases
the software could encounter in real-world use.

Given the central role software plays in modern scientific discovery, it deserves
an equally rigorous and systematic approach. One possible way to towards achieving
more confidence in software correctness is through property-based testing, which
brings together two essential qualities:

Rigor: Property-based testing systematically explores a wide and often surprising range
of inputs, not just a few, hand-picked cases.

Reasoning: It encourages developers to express general behavioral properties,
promoting abstraction, logical thinking, and clarity about the software’s intended behavior.

## So What is Property-Based Testing?

Property-based testing emerged in ... Haskell .. Quickcheck

# The hypothesis library

[REWORD] The [hypothesis](https://hypothesis.works/) library is a property-based testing framework for Python. It is the most widely used property-based testing library in the Python ecosystem and is inspired by Haskell's QuickCheck. Hypothesis allows developers to define properties that their code should satisfy, and it automatically generates a wide range of test cases to verify these properties.

We'll introduce the fundamental concepts of property-based testing on the example of a simple multiplication function. This will help us understand how the main concepts of property-based testing work.

In [53]:
def multiply(a, b):
    """Returns the product of two numbers."""
    return a * b

Before diving into property-based testing, let’s begin (as we should) with some simple unit tests.

We’ll check whether our multiply function satisfies the associativity property, i.e., that `(a*b)*c == a*(b*c)` holds true.

Let’s write a few test cases to verify this property for our multiply function:


In [105]:
def test_associative():
    """Test that multiplication is associative."""
    assert multiply(multiply(2, 3), 4) == multiply(2, multiply(3, 4))
    assert multiply(multiply(-1, 5), 6) == multiply(-1, multiply(5, 6))
    assert multiply(multiply(0, 10), 20) == multiply(0, multiply(10, 20))
    assert multiply(multiply(1, 1), 1) == multiply(1, multiply(1, 1))
    assert multiply(multiply(2.5, 4.0), 1.5) == multiply(2.5, multiply(4.0, 1.5))
    assert multiply(multiply(-3.5, 2.0), 3.0) == multiply(-3.5, multiply(2.0, 3.0))
    assert multiply(multiply(1e20, 1e30), 1e40) == multiply(1e20, multiply(1e30, 1e40))
    assert multiply(multiply(0.0, 0.0), 0.0) == multiply(0.0, multiply(0.0, 0.0))
    assert multiply(multiply(2.765, 4), 3.14) == multiply(2.765, multiply(4, 3.14))
    assert multiply(multiply(2.81, 3.76), 1.61) == multiply(2.81, multiply(3.76, 1.61))
    print("All tests passed!")

test_associative()

All tests passed!



Running the following code should pass all tests, confirming that our multiply function appears to be associative.

**But wait!** 

Floating-point multiplication isn’t actually associative. So what’s going on here?

This example highlights a few common pitfalls of unit testing: while it's certainly a necessary first step toward correctness, it can be tedious, repetitive, and misleading. Unit tests often overlook subtle numerical behaviors and edge cases, giving us a false sense of confidence.

Can we do better? Yes. We can use property-based testing to explore the behavior of our multiplication function more rigorously and systematically, uncovering issues that traditional unit tests might miss.

In [79]:
import numpy as np

In [81]:
np.inf * 0

nan

In [83]:
1 * np.inf

inf

Now let's test out several properties of this multiplication function using property-based testing.
First, we'll confirm that the multiplication function is commutative, meaning that the order of the operands does not affect the result.

The two main components of property-based testing are:
- *strategy*: This is a way to define the types of inputs that will be generated for testing.
- *given*: This is a decorator that allows you to define a property that should hold true for a wide range of inputs.

Let's start by importign these two components from the hypothesis library:

In [54]:
from hypothesis import strategies as st
from hypothesis import given

In [55]:
@given(st.integers(), st.integers())
def test_commutative(a, b):
    """Test that integer multiplication is commutative."""
    assert multiply(a, b) == multiply(b, a)

test_commutative()

Running the above cell produces no output, which means that no assertion
was violated—in other words, the property held true for all generated inputs.
To confirm that Hypothesis is indeed generating random inputs and evaluating
the property, we can add a print statement before the assertion. Also,
by default, Hypothesis runs 100 test cases per property. To make the output
easier to follow, we’ll reduce this number to 10 using the @settings decorator.

In summary, we’ll:
- Use `@settings(max_examples=10)` to limit the number of generated cases
- Add `print(f"Testing a={a}, b={b}")` to display the generated inputs

Let’s update the test accordingly.

In [62]:
from hypothesis import settings

@settings(max_examples=10)  # Run with 10 test cases instead of default (100)
@given(st.integers(), st.integers())
def test_commutative(a, b):
    """Test that integer multiplication is commutative."""
    print(f"Testing a={a}, b={b}")
    assert multiply(a, b) == multiply(b, a)

test_commutative()

Testing a=0, b=0
Testing a=-8185, b=0
Testing a=-8185, b=-31381
Testing a=17881, b=-2461788500594381115
Testing a=133495542271790161, b=-37
Testing a=-17478, b=-86851563182348660229424653882970253239
Testing a=-22843, b=-11046
Testing a=-97, b=3274
Testing a=-96, b=-5953898863333731411
Testing a=-1252919588, b=-3307258136486730639


Run the above cell multiple times to see how the generated inputs change.
This demonstrates that Hypothesis is indeed generating random inputs for the property.

Now let's test the same property for floating-point numbers. To do so, we simply change
the input strategy to `st.floats()`:

In [71]:
@given(st.floats(), st.floats())
def test_commutative(a, b):
    """Test that multiplication is commutative for floats."""
    assert multiply(a, b) == multiply(b, a)

test_commutative()

AssertionError: 

This time, you should receive an assertion error, indicating that the property
does not hold for floating-point numbers. 

In [ ]:
@given(st.floats(), st.floats())
def test_associative(a, b):
    """Test that multiplication is associative."""
    assert multiply(multiply(a, b), 2) == multiply(a, multiply(b, 2))

In [21]:
test_associative()

AssertionError: 

In [ ]:
# Now, let's run the test
if __name__ == "__main__":
    import pytest
    pytest.main([__file__])

In [22]:
from hypothesis.extra.numpy import arrays as st_arrays

In [8]:
@given(st_arrays(dtype=float, shape=(10,)))
def d_dx(q, dx):
    """Centered finite difference approximation of the derivative."""
    return (q[2:] - q[:-2]) / (2 * dx)